In [1]:
import pandas as pd
import numpy as np
import joblib
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [ ]:
lgbm_model_path = 'best_models/best_lgbm_selected.pkl'
xgb_model_path = 'best_models/best_xgb.pkl'

lgbm_model = joblib.load(lgbm_model_path)
xgb_model = joblib.load(xgb_model_path)
print("Models loaded successfully.")

train = pd.read_csv('./dataset/processed-data/final_train.csv', index_col=False)
X_test = pd.read_csv('./dataset/processed-data/final_test.csv', index_col=False)
X_train = train.drop(columns=['LOG_RESALE_PRICE'])
y_train = train['LOG_RESALE_PRICE']

In [8]:
best_lgbm = joblib.load("./best_models/best_lgbm.pkl")
lgbm_impt = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_lgbm.feature_importances_
})
top_feat = lgbm_impt.sort_values(by='Importance', ascending=False).head(30)
X_train_lgbm = X_train[top_feat['Feature'].tolist()]
X_test_lgbm = X_test[top_feat['Feature'].tolist()]

In [10]:
# train on the full dataset
lgbm_model = LGBMRegressor(**lgbm_model.get_params())
lgbm_model.fit(X_train_lgbm, y_train)
xgb_model = XGBRegressor(**xgb_model.get_params())
xgb_model.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1936
[LightGBM] [Info] Number of data points in the train set: 162570, number of used features: 30
[LightGBM] [Info] Start training from score 13.099900


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.6292054996882723
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [11]:
lgbm_log_predictions = lgbm_model.predict(X_test_lgbm)
xgb_log_predictions = xgb_model.predict(X_test)
# ensemble the models by averaging their log predictions
ensemble_log_predictions = (lgbm_log_predictions + xgb_log_predictions) / 2
final_original_predictions = np.exp(ensemble_log_predictions)

In [12]:
submission_df = pd.DataFrame({
    'Id': [i for i in range(len(final_original_predictions))],
    'Predicted': final_original_predictions
})
submission_df.to_csv('predictions_fulltrain.csv', index=False)